***

***

# Sesión 4 de SM: Procesamiento de audio con Python <img src="imgs/logo_uex.png" alt="missing" width=22>

Práctica desarrollada por Andres J. Sanchez-Fernandez (sfandres@unex.es) y Juan M. Haut (juanmariohaut@unex.es) para la asignatura Sistemas Multimedia de la Universidad de Extremadura.

***

***

## Organización de la clase

Fecha de la sesión planificada: 05/03/2024.

<table>
  <tr>
    <th>Turno</th>
    <th>Tiempo (')</th>
    <th>Tarea</th>
  </tr>
  <tr>
    <td>Profesor</td>
    <td>10</td>
    <td>Introducir el contenido de este notebook.</td>
  </tr>
  <tr>
    <td>Alumnos</td>
    <td>20</td>
    <td>Acceder al <a href="https://github.com/sfandres94/uex-audiopy2">GitHub</a> de la práctica y leer detenidamente todos los conceptos que se explican.</td>
  </tr>
  <tr>
    <td>Alumnos</td>
    <td>&#8734;</td>
    <td>Realizar el ejercicio de clase de análisis y procesamiento de ondas de audio con Python3.</td>
  </tr>
</table>

## Objetivos

Los objetivos de esta práctica son los siguientes:
<ol>
    <li>Entender lo que es una <b>onda de sonido</b> y cómo se realiza el <b>proceso de adquisición</b> de la misma.</li>
    <li>Comprender la importancia de los conceptos básicos que caracterizan una onda:
        <ul>
            <li>La <b>frecuencia de muestreo</b> (<em>sample rate</em>).</li>
            <li>La <b>profundidad de bits</b> (<em>bit depth</em>).</li>
            <li>El <b>ancho de banda</b> (<em>bandwidth</em>).</li>
            <li>La <b>tasa de bits</b> (<em>bitrate</em>).</li>
        </ul>
    <li>Conocer la teoría de muestreo de Nyquist y el problema de <em>Aliasing</em>.</li>
    <li>Conocer la diferencia entre gráfica de onda en el <b>dominio del tiempo</b> vs <b>de la frecuencia</b> (Transformada de Fourier).</li>
    <li>Aprender el <b>concepto de energía del espectrograma</b> y realizar la <b>compresión de audio</b> en base al mismo.</li>
</ol>

## Bibliografía consultada

Información consultada para el desarrollo de esta práctica:
<ul>
    <li><em>RouteNote Blog: Understanding sample rates in digital audio by Connor Edney</em> <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Link]</a></li>
    <li><em>MiniTool: What Is Audio Sample Rate & How to Change Sample Rate of Audio by Cora</em> <a href="https://videoconvert.minitool.com/video-converter/audio-sample-rate.html">[Link]</a></li>
    <li>Repositorios de Github:
        <ul>
            <li>Cómo graficar espectrogramas de Audios en Python <a href="https://www.kaggle.com/code/joeportilla/c-mo-graficar-espectrogramas-de-audios-en-python/notebook">[Link]</a></li>
            <li><em>Simple audio compression with Python</em> <a href="https://github.com/jmpion/simple-audio-compression-with-python/blob/master/simple-audio-compression.ipynb">[Link]</a></li>
        </ul>
    </li>
</ul>

***

***

## [Práctica anterior] Primeros pasos con audio: sonido mono vs estéreo

<center><img src="imgs/mono_vs_stereo_diagram.jpg" alt="missing" width=500></center>

### Importar librerías y módulos de Python

<b>Nota</b>: Estas no son las únicas librerías de Python con las que se pueden trabajar para el procesamiento de archivos de audio (otro ejemplo podría ser <a href="https://librosa.org/doc/latest/index.html">Librosa</a>). Si alguien está familiarizado con otras librerías puede utilizarlas.

Importamos las librerías/módulos específicos de la siguiente forma.

In [ ]:
# Importacion.
# import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

### Especificar directorios de entrada y salida

Aquí definimos los directorios donde guardaremos los audios con los que vamos a trabajar, así como dónde se van a guardar aquellos que generamos a lo largo de la práctica.

In [ ]:
# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio', '_input'))  # cambiar '_input' por 'examples'
audio_output_path = os.path.join(cwd, os.path.join('audio', '_output'))
print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

### Cargar el archivo de audio

Diferencias entre formatos de archivo para almacenar audio digital.

<ul>
    <li><b>.wav</b>: Archivo de audio sin comprimir (máxima calidad y gran tamaño de archivo). Típicamente utilizado en edición de audio debido a su fidelidad.</li>
    <li><b>.mp3</b> (por ejemplo): Archivo de audio comprimido (con pérdidas pero menor tamaño). Ampliamente usado.</li>
</ul>

Cargamos el archivo de audio .wav en este caso.

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join(audio_input_path, 'sample1_stereo.wav')
# audio_data, sample_rate = librosa.load(filename, sr=None, mono=False)
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

Vamos a escucharlo. Para que esto se haga correctamente, hay que indicarle la frecuencia de muestreo (veremos más adelante qué es).

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo.

### Mostrar principales características de la onda

Vamos a mostrar la información. Nota: es audio estereo (dos canales).

In [ ]:
# Mostrar informacion (sonido estéreo).
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}')
print(f'- 1º canal:   {audio_data[:5, 0]}...')
print(f'- 2º canal:   {audio_data[:5, 1]}...')
print(f'- Resolucion: {type(audio_data[0,0])}\n')

Ahora, por simplificación, vamos a calcular la media por canal para obtener un sonido mono.

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')
print(f'- Canal unico:  {new_data_mono[:5]}...')

# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')

Vamos a guardarlo.

In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'sample1_mono.wav'),
    rate=sample_rate,
    data=new_data_mono
)

Vamos a escucharlo de nuevo.

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

 Se nota que ahora es sonido mono (sobre todo si utilizais cascos).

<ul>
    <li><b>Mono</b>: se escucha lo mismo por el auricular derecho que por el izquierdo.</li>
    <li><b>Estéreo</b>: no se escucha el mismo sonido por ambos canales, sino que se notan variaciones entre los dos.</li>
</ul>

Vamos a ver las diferencias en tamaño de cada archivo.

In [ ]:
!ls -sh audio/_input/sample1_stereo.wav
!ls -sh audio/_output/sample1_mono.wav

Como podemos ver el tamaño se ha reducido a la mitad (manteniendo el la frecuencia de muestreo). Mostramos por pantalla la frecuencia de muestreo (*sample rate*) del archivo de audio: 

In [ ]:
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz\n')

Muy bien la diferencia entre sonido estéreo y mono pero:

*¿cómo se adquiere esta onda de audio?*, *¿qué significa esta frecuencia de muestreo?*, *¿para que sirve la transformada de Fourier?*, *¿para qué queremos comprimir una onda?*, *etc*.

Todo esto y más lo veremos a continuación.

***

***

## [Nuevo] Teoría: Toma de muestras de audio

### Definición de muestra

Una muestra es una <b>medición instantánea de una onda sonora</b>: una medida de la amplitud de la onda (intensidad de la señal).

Un sistema digital, como una interfaz de audio, toma miles de muestras individuales que registran la amplitud de la onda sonora (cambios en la presión del aire) para reconstruirla digitalmente.

<figure>
    <img src="imgs/wavecycle.jpg"
         alt="missing"
         width=350>
    <figcaption><em><b>Figura</b>: Ejemplo de una onda sonora simplificada <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Ref]</a>.</em></figcaption>
</figure>

Las ondas sonoras son ondas continuas y están formadas por ciclos de onda individuales. Por tanto, <b>una muestra es una medición de la amplitud de un ciclo de onda individual</b>.

Entonces, nuestra interfaz puede convertir la onda sonora en datos binarios, interpretables por nuestro ordenador. Ahora bien, ¿cada cuánto tomamos una muestra?

### Frecuencia de muestreo (*sample rate*)

#### Definición

La frecuencia de muestreo equivale al <b>número de muestras que tomamos por segundo</b> y, por tanto, a la velocidad a la que lo hacemos.

En la práctica, necesitamos tomar miles de muestras de nuestra grabación por segundo si queremos que nuestra señal digital suene fiel a nuestra onda sonora original.

<figure>
    <img src="imgs/sample_rate.jpg"
         alt="missing"
         width=500>
    <figcaption><em><b>Figura</b>: Explicación visual de la frecuencia de muestreo de una onda de audio. En el primer ejemplo (A), el resultado digital es deficiente
        porque las muestras no son lo bastante frecuentes. En el segundo (B), el resultado es mucho mejor y más suave. Ahora bien, es en el tercer ejemplo (C) donde el 
        resultado digital es tan suave como el audio original al tomarse suficientes muestras <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Ref]</a>.</em>
    </figcaption>
</figure>

A mayor frecuencia de muestreo, capturamos mayor cantidad de detalles de la onda original.

La unidad de la frecuencia de muestreo es el Hercio (Hz), que equivale a un periodo de muestreo T = 1s, aunque normalmente se utiliza la unidad derivada kHz (1 kHz = 1 000 Hz).

#### Valores más comunes de frecuencia de muestreo

Una de las frecuencias de muestreo de audio **más común actualmente es de 44,1 kHz**, esto significa que se toman **44 100 muestras de audio por segundo**. Las más habituales son:
<ul>
    <li>8 kHz (8 000 muestras/s) es la frecuencia de muestreo de audio utilizada por los teléfonos, ya que es suficiente para el habla humana.</li>
    <li>32 kHz es ampliamente utilizada por las videocámaras digitales MiniDV, DAT, cintas de vídeo, etc.</li>
    <li>44,1 kHz es la frecuencia de muestreo de audio estándar para el audio de CD, y también se utiliza para el audio MPEG-1.</li>
    <li>48 kHz es la frecuencia de muestreo que suelen utilizar los equipos de vídeo digital, DVD, TV digital, películas y la mayoría de los equipos de audio profesionales.</li>
    <li>50 kHz es la frecuencia de muestreo utilizada principalmente por los grabadores digitales comerciales.</li>
    <li>88,2 kHz es la frecuencia de muestreo utilizada por algunos equipos de grabación profesionales cuando se apunta a un CD para realizar grabaciones de alta resolución.</li>
    <li>96 kHz es el doble del estándar de 48 kHz y se utiliza principalmente en DVD-Audio, pistas de audio de Blu-ray Disc, pistas de audio de DVD de alta definición, etc. Y también está disponible en algunos equipos profesionales de grabación y producción de audio.</li>
    <li>192 kHz empleada principalmente en Hi-Res audio junto con la de 96.
</ul>

#### Pregunta sencilla: a mayor frecuencia de muestreo, ¿mayor calidad?

La solución es también sencilla: **¡Sí!**

Al aumentar el número de muestras que se toman por segundo, tenemos una mayor resolución, y con ello una mayor calidad del audio.

Sin embargo, no todo es bueno: Al grabar con una frecuencia de muestreo más alta, se crea un archivo de audio también mayor. Es decir, como se toman más muestras, se necesita más espacio en disco para almacenarlas y un procesador con mayor potencia para su procesamiento.

Ya no es solo el mayor espacio ocupado en disco, sino que un archivo de audio grande es contraproducente para transmitirlo por internet, por ejemplo, en plataformas de streaming tales como Spotify, Amazon Music, etc.

#### Caso real: ¿A qué valor fijamos la frecuencia de muestreo?

Posible respuesta: <b>Teorema de muestreo de Nyquist-Shannon</b>.

La teoría de Nyquist establece que *necesitamos una frecuencia de muestreo igual al doble de la frecuencia más alta de una señal para capturar todas las frecuencias de la misma*.

Este hecho se debe a que un ciclo de onda singular siempre tiene un valor de amplitud negativo y otro positivo. Estos valores son la medida de la intensidad de la señal de los ciclos (amplitud). Y para hallar la longitud de onda de cada ciclo de onda---que determina sus frecuencias---se deben muestrear las amplitudes positiva y negativa de cada ciclo.

En consecuencia, <b>debemos muestrear cada ciclo dos veces (como mínimo)</b> si queremos que nuestra señal digital tenga la frecuencia correcta.

<figure>
    <img src="imgs/nyquist.png"
         alt="missing"
         width=450>
    <figcaption><em><b>Figura</b>: Onda sonora muestreada según el teorema de Nyquist <a href="https://www.homestudioproductions.com/frecuencia-de-muestreo-profundidad-de-bits-que-es/">[Ref]</a>.</em>
    </figcaption>
</figure>



#### Cuando el muestreo sale mal: *Aliasing*

La teoría de muestreo de Nyquist nos dice que tomar menos de 2 muestras por ciclo conduce a una reconstrucción digital incorrecta de nuestra señal: un "alias" no deseado del original. Por tanto, se denomina *aliasing*, o solapamiento, al <b>efecto que causa que señales continuas distintas sean indistinguibles cuando se muestrean digitalmente a causa de una frecuencia de muestreo demasiado baja</b>. Cuando esto sucede, la señal original no puede ser reconstruida de forma unívoca a partir de la señal digital.

<p style="border-width:1.25px; border-style:solid; border-color:#0000FF; background-color:#EEEEEE; padding:1em;">
    <b>Ejemplo:</b> Vamos a muestrear la siguiente señal siguiendo el criterio de Nyquist:
</p>

<figure>
    <img src="imgs/nyquist_sample_rate_perfect.jpg"
         alt="missing"
         width=400>
    <figcaption><em><b>Figura</b>: Onda sonora muestreada con una frecuencia de muestro que cumple con el teorema de Nyquist <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Ref]</a>.</em>
    </figcaption>
</figure>
<br>
<p style="border-width:1.25px; border-style:solid; border-color:#0000FF; background-color:#EEEEEE; padding:1em;">
    Podemos observar como la señal reconstruida (3) es similar a la original (1) ya que hemos tomado suficientes muestras (2).<br><br>
    Ahora bien, si aumentamos la frecuencia de la señal original sin aumentar el número de muestras ocurre lo siguiente:
</p>
    
<figure>
    <img src="imgs/too_low_a_sample_rate_poor.jpg"
         alt="missing"
         width=400>
    <figcaption><em><b>Figura</b>: Onda sonora muestreada con una frecuencia de muestro menor que la necesaria según el teorema de Nyquist <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Ref]</a>.</em>
    </figcaption>
</figure>
<br>
<p style="border-width:1.25px; border-style:solid; border-color:#0000FF; background-color:#EEEEEE; padding:1em;">
    Ahora podemos ver que, a pesar de que la nueva señal (4) es muy diferente a la que vimos anteriormente (1), el resultado de la señal reconstruida (6) es el mismo que en el caso anterior (3).<br><br>
    <b>Entonces concluímos que se ha producido <i>aliasing</i>.</b>
</p>

***

***

## Mas teoría: Características de grabación de audio digital

### Profundidad de bits (*bit depth*)

Una vez que hemos capturado la onda como se ha explicado, hay que almacenar la información en forma de bits.

La profundidad de bits determina <b>cuántos bits disponibles hay para medir la onda sonora</b> en primer lugar, y luego para que almacenemos nuestras muestras en bytes digitales.

<figure>
    <img src="imgs/adc.jpg"
         alt="missing"
         width=500>
    <figcaption><em><b>Figura</b>: Conversión Analógica Digital (ADC) <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Ref]</a>.</em>
    </figcaption
</figure>

### Ancho de banda (*bandwidth*)

En otras palabras, <b>la profundidad de bits y la frecuencia de muestreo trabajan juntas para darnos el ancho de banda total de nuestra grabación digital</b>.

\begin{equation}
\text{Profundidad de bits (bit depth)}+\text{frecuencia de muestreo (sample rate)}=\text{ancho de banda (bandwidth)}
\end{equation}

Esto significa que el ancho de banda total <b>define la precisión de nuestra señal digital con respecto a la grabación original</b>. Más ancho de banda implica una reproducción más exacta.

<figure>
    <img src="imgs/bit_depth.jpg"
         alt="missing"
         width=600>
    <figcaption><em><b>Figura</b>: La frecuencia de muestreo y la profundidad de bits forman el ancho de banda de audio total. La cantidad de bits disponibles se corresponde con el número de valores de amplitud digital a los que podemos asignar nuestras muestras <a href="https://routenote.com/blog/what-is-sample-rate-in-audio/">[Ref]</a>.</em>
    </figcaption
</figure>

### Suele llevar a confusión: Frecuencia de muestreo vs tasa de bits (*bitrate*)

Como se ha explicado, la frecuencia de muestreo representa cuántas muestras de la onda sonora original tomamos por segundo. Aunque tenemos otra variable también: el flujo o tasa de bits, que determina el tamaño del archivo de audio digital. La tasa de bits es un <b>cálculo matemático del tamaño de los archivos digitales en megabytes por segundo</b> (Mbps)---no confundir con la profundidad de bits.

Para calcular la tasa de bits de tu grabación digital, puedes utilizar el siguiente cálculo:

\begin{equation}
\text{bitrate}\;(Mbps) = f_s \cdot f_d \cdot n_c
\end{equation}

donde $f_s$ es la frecuencia de muestreo, $f_d$ es la profundidad de bits y $n_c$ es el número de canales de la grabación.

En base a todo esto, se puede decir que la tasa de bits determina el <b>número de bits que el ordenador debe procesar por segundo para reproducir la grabación de audio digital de la forma prevista</b>.

<p style="border-width:1.25px; border-style:solid; border-color:#0000FF; background-color:#EEEEEE; padding:1em;">
    <b>Ejemplo:</b> Supongamos que grabamos un riff de guitarra con una frecuencia de muestreo de 44,1 kHz y una profundidad de bits de 24 bits. Entonces tendríamos una tasa de bits de:<br><br>
    $44\,100\,(Hz)\cdot 24\,(bits) \cdot 1\,canal = 1\,058\,400\,(bps) = 1,1\,(Mbps)$<br><br>
    Después de calcular la tasa de bits de tu grabación, multiplícala por el número de segundos de tu grabación si quieres saber el tamaño total del archivo. Digamos que grabamos durante 20 segundos, nos quedaría:<br><br>
    $1,1\,(Mbps) \cdot 20\,(s) = 22\,Mb$
</p>





***

***

## Práctica: Compresión de audio

### Cargar audios

Vamos a cargar otros dos archivos de audio adquiridos a distintas frecuencias de muestreo: 48 y 24 kHz.

In [ ]:
# Cargamos los archivos de audio.
sample_rate_48, audio_data_48 = wavfile.read(filename=os.path.join(audio_input_path, 'sample_48kHz.wav'))
sample_rate_24, audio_data_24 = wavfile.read(filename=os.path.join(audio_input_path, 'sample_24kHz.wav'))

# Otro ejemplo sería: audio_data, sample_rate = librosa.load(filename)

print(f'{audio_data_48.shape}\n')  # Audio mono
print(f'{audio_data_24.shape}\n')  # Audio mono

Vamos a escuchar los audios.

In [ ]:
IPython.display.Audio(audio_data_48, rate=sample_rate_48)

In [ ]:
IPython.display.Audio(audio_data_24, rate=sample_rate_24)

Tenemos los siguientes tamaños de archivo.

In [ ]:
!ls -sh audio/_input/sample_48kHz.wav
!ls -sh audio/_input/sample_24kHz.wav

Como se puede ver y es lógico, la frecuencia de muestreo también afecta al tamaño del archivo de audio

### Gráficas características

#### Dominio del tiempo

In [ ]:
ampl_values_48 = len(audio_data_48)
ampl_values_24 = len(audio_data_24)
print(f'Número de muestras del audio con fs=48 kHz (valores de amplitud): {ampl_values_48}')
print(f'Número de muestras del audio con fs=24 kHz (valores de amplitud): {ampl_values_24}')

In [ ]:
# Construimos el array para el eje x que representa el tiempo de la grabación.
# Tiene la forma: np.arange(Vi, Vf, P). Explicado a continuación.
t1 = np.arange(0, ampl_values_48/sample_rate_48, 1/sample_rate_48)
t2 = np.arange(0, ampl_values_24/sample_rate_24, 1/sample_rate_24)

Vamos a construir el eje x con un array que tenga como:
<ul>
    <li>Valor inicial ($V_i$): $$V_i=0\;s$$</li><br>
    <li>Valor final ($V_f$): $$V_f = \frac{N}{f_s}$$<br> donde $N$ representa el número total de muestras (valores de amplitud) y $f_s$ es la frecuencia de muestreo en Hz (número total de muestras capturadas por segundo). Esto nos da la duración total de la grabación en segundos.</li><br>
    <li>Paso o <i>step</i> ($P$): $$P=T=\frac{1}{f_s}$$<br>donde $T$ se refiere al periodo de la señal, es decir, el tiempo transcurrido entre dos puntos equivalentes de la onda.</li><br>
<ul>


In [ ]:
print(t1)
print(t2)

In [ ]:
# Creamos la figura.
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Solo mostramos las primeras 50 muestras de amplitud (por claridad).
end = 50

# Señal a 48 kHz.
ax[0].plot(t1[:end], audio_data_48[:end], marker='X')
ax[0].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate_48} Hz')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

# Señal a 24 kHz.
# Utilizamos ratio para ajustar el eje x de ambas gráficas
# ya que la fs es menor en esta señal.
ratio = sample_rate_48 / sample_rate_24  
ax[1].plot(t2[:int(end/ratio)], audio_data_24[:int(end/ratio)], c='tab:red', marker='X')
ax[1].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate_24} Hz')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

# Mostramos la figura.
plt.tight_layout()
plt.show()

Cada cruz que se ve en las gráficas representa una muestra recogida del audio. Podemos ver que la primera onda tiene un mayor número de muestras recogidas para el mismo periodo de tiempo (mismos valores en el eje x). Por tanto, la primera onda tiene también una precisión mayor que la segunda al tener el doble de frecuencia de muestreo, siendo entonces más fiel a la onda sonora original (fuente).

#### Dominio de la frecuencia: Transformada de Fourier (FFT) 

##### Teoría

Vamos a descomponer la señal en vez de en el dominio del tiempo (eje x: tiempo en segundos) en el dominio de la frecuencia (eje x: frecuencia en Hz). A continuación se presentan dos ejemplos animados del funcionamiento de la Transformada de Fourier:

<figure>
    <img src="imgs/Fourier_1.gif"
         alt="missing"
         width=400>
    <figcaption><em><b>Figura animada</b>: Ejemplo 1 de la transformada de Fourier <a href="https://en.wikipedia.org/wiki/File:Fourier_transform_time_and_frequency_domains.gif">[Ref]</a>.</em>
    </figcaption
</figure>

<figure>
    <img src="imgs/Fourier_2.gif"
         alt="missing"
         width=450>
    <figcaption><em><b>Figura animada</b>: Ejemplo 2 de la transformada de Fourier <a href="https://commons.wikimedia.org/wiki/File:Fourier_synthesis_square_wave_animated.gif">[Ref]</a>.</em>
    </figcaption
</figure>

##### Análisis de Fourier

La "Transformación rápida de Fourier" (FFT para abreviar) <b>descompone una señal en sus componentes espectrales individuales, proporcionando información sobre su composición.</b> Este es un método importante de medición en la tecnología de audio.

<figure>
    <img src="imgs/Fourier_3.png"
         alt="missing"
         width=450>
    <figcaption><em><b>Figura</b>: Transformada de Fourier <a href="https://luisgarciamillan.es/transformada-de-fourier/">[Ref]</a>.</em>
    </figcaption
</figure>

In [ ]:
# La longitud del array de datos y el
# sample rate (frecuencia de muestreo).
n = len(audio_data_48)
Fs = sample_rate_48

# Working with stereo audio, there are two channels in the audio data.
# Let's retrieve each channel seperately:
# ch1 = np.array([data[i][0] for i in range(n)]) #channel 1
# ch2 = np.array([data[i][1] for i in range(n)]) #channel 2
# We can then perform a Fourier analysis on the first
# channel to see what the spectrum looks like.

# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(audio_data_48)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

### Energia del espectrograma y frecuencia de corte

Ahora vamos a definir una frecuencia umbral $f_0$ por la que cortar el espectro, es decir, <b>solo nos quedaremos con aquellas frecuencias que esten por debajo de este valor para el archivo de audio comprimido</b>.

Con este fin, definimos el parámetro <b>epsilon</b> $\epsilon \in (0, 1)$ que <b>representa la parte de la energía del espectro que NO conservaremos</b> (la integral con respecto a la frecuencia).

En esta práctica, a modo de ejemplo, seleccionamos un $\epsilon=10^{-5}$ para quitar únicamente una parte, podéis jugar con este valor como queráis para ver el resultado. De manera intuitiva, a medida que aumenta su valor, se mantienen menos frecuencias por lo que la calidad del audio es peor aunque el tamaño del archivo de salida es más reducido. Por lo que tenemos que buscar un buen balance en este sentido.

In [ ]:
# Definimos diferentes epsilons: la parte de
# la energia del espectro que NO conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (ID VARIANDO ESTE VALOR Y MIRAD LA GRÁFICA).
eps = eps[1]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

### Compresión del archivo

#### Reducción de la resolución de muestreo (*downsampling*)

<b>Para reducir el tamaño del archivo de audio</b>, lo que vamos a hacer es aplicar <b><em>downsampling</em></b>. Vamos a definir el factor de <em>downsampling</em> $D$, el cual utilizaremos para quedarnos únicamente con la secuencia de valores ($\hat{x}$) del audio original ($x$) como sigue:

\begin{equation}
\hat{x}[i] = x[D\cdot i]
\end{equation}

Esta equación quiere decir que <b>la secuencia de audio resultante estará compuesta por aquellos elementos elementos situados en las posiciones múltiplo de $D$ (<em>slicing</em>)</b>. Para saber qué valor debemos utilizar para el <em>downsampling</em>, una opción es la de calcular $D=\frac{f_s}{f_0}$, por lo que la nueva frecuencia de muestreo (<em>sample rate</em>) estaría definida por $\hat{f}_s=\frac{f_s}{D}$.

Gracias a todo este proceso, las frecuencias que constituyen la parte principal del espectro quedarán por debajo de la nueva frecuencia de muestreo.

In [ ]:
# Definimos los nombres de los audios comprimidos.
wav_compressed_file = "sample_48kHz_compressed.wav"

# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = audio_data_48[::D]

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

Vamos a escucharlo.

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

Consejo: probad a cambiar la variable `eps` y ejecutar de nuevo las celdas para ir viendo como se deteriora la calidad al aumentar el factor de *downsampling*.

Después de la compresión, el tamaño final será menor:

In [ ]:
!ls -sh audio/_output/sample_48kHz_compressed.wav

#### Espectrograma

El espectrograma es una representación visual que muestra las diferentes variaciones de la frecuencia (eje y) y la intensidad del sonido (color) a lo largo del tiempo (eje x).

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(audio_data_48, NFFT=1024, Fs=sample_rate_48, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

Podemos observar como la resolución se ha reducido, aunque aún se pueden apreciar características similares en ambos espectrogramas.

***

***

In [ ]:
# # Ignorar: Codigo para reducir un archivo de audio truncando

In [ ]:
# # Input data.
# audio_examples_input_path = os.path.join(cwd, os.path.join('audio', 'examples'))
# filename = os.path.join(audio_examples_input_path, 'interstellar.wav')
# sample_rate_v0, audio_data_v0 = wavfile.read(filename)
# print(f'Shape of the input audio data: {audio_data_v0.shape}\n')

In [ ]:
# # Slicing with stride (stereo).
# new_data_v0 = audio_data_v0[:10856495//4, :]

# # Writing to a wav file.
# wavfile.write(
#     filename=os.path.join(audio_examples_input_path, 'interstellar2.wav'),
#     rate=sample_rate_v0,
#     data=new_data_v0
# )

# # Loading the new audio data.
# filename = os.path.join(audio_examples_input_path, 'interstellar2.wav')
# sample_rate_v1, audio_data_v1 = wavfile.read(filename)
# print(f'Shape of the input audio data min: {audio_data_v1.shape}')
# print(f'Reduction (%): {int((len(audio_data_v0)-len(audio_data_v1))*100/len(audio_data_v0))}')

In [ ]:
# IPython.display.Audio(audio_data_v1.T, rate=sample_rate_v1)